In [1]:
!pip install nltk

In [3]:
# import pandas as pd
# from nltk.corpus import stopwords
# import nltk
import re
# from tensorflow.keras.utils import to_categorical

In [67]:
def split_special_character( word: str) -> str:
        # add white space between normal word and special character.

        tail_special = '""''v@_!#$%^&*()<>?/\|}{~:;[],.+-\n'
        head_special = '@_!#$%^&*()<>?/\|}{~:;[],.+-'

        for i in reversed(range(len(word))):
            if word[i] not in tail_special:
                break
        
        if i != len(word) - 1:
            word = word[0: i + 1] + " " + word[i + 1:]
       

        for i in range(len(word)):
            if word[i] not in head_special:
                break
          
        if i != 0:
            word = word[0: i] + " " + word[i:]            
            return word
        return word

In [71]:
split_special_character("(@manh].vvvv")

'(@ manh ].vvvv'

In [60]:
" @(manh)@gmail.com tôi có ất nhiều (@manh)"[34+1:35+1]

'('

# Datasets

## Clean Data Funcion

In [ ]:
import re
import unicodedata
from typing import List
import numpy as np
import string
from pyvi import ViTokenizer, ViPosTagger

def split_special_character(word: str, pos_head=True):
    # add white space between normal word and special character.

    tail_special = '""''v@_!#$%^&*()<>?/\|}{~:;[],.+-\n'

    for i in reversed(range(len(word))):
        if word[i] not in tail_special:
            break
    try:
      if i != len(word) - 1:
          word = word[0: i + 1] + " " + word[i + 1:]
    except:
      print('ERROR word = {}'.format(word))
      return word
    if pos_head is True:
        head_special = '@_!#$%^&*()<>?/\|}{~:;[],.+-'

        for i in range(len(word)):
            if word[i] not in head_special:
                break
        
        try:
          if i != 0:
              word = word[0: i] + " " + word[i:]
        except:
          print('ERROR word = {}'.format(word))
         
          return word
    return word


def replace_special_character(word: str):

    characters = {'™': '*', '‘': "'", '®': 'x', '×': 'x', '😀': 'x', '‑': '-',
                  '́': 'x', '—': ' - ', '̣': 'x', '–': '-', '`': "'", '“': '"', '̉': 'x',
                  '’': "'", '̃': 'x', '\u200b': 'x', '̀': 'x', '”': '"', '…': '...',
                  '\ufeff': 'x', '″': '"'}
    try:
      for c in characters:
          if c in word:
              word = word.replace(c, characters[c])
    except:
      print('ERROR word = {}'.format(word))
      
    return word


def preprocess_sen(sentence: str) -> str:

    # sentence = preprocess_stopword(sentence, list_stopwords)
    words = sentence.split(' ')
    norm_words = []

    for i in range(len(words)):
        word = words[i]
        word = unicodedata.normalize('NFC', word)

        word = split_special_character(word)
        word = replace_special_character(word)

        norm_words.append(word)
    
    norm_text = " ".join(norm_words)

    return norm_text


def split_sentence(text: str):

    text = text.replace('\r', '')
    text = text.replace('\n', '. ')
    text += ' '

    sentences = text.split('. ')
    sentences = [sen.strip(' .') for sen in sentences]
    sentences = [sen for sen in sentences if sen != '']
    sentences = [sen + '.' for sen in sentences]

    return sentences


def preprocess_stopword(sentence: str, list_stopwords: list) -> str:

    parts = []
    special_char = '@_!#$%^&*()<>?/\|}{~:;[],.+-'

    for i in special_char:
      sentence =  sentence.replace(i, ' ')

    sentence = re.sub(' +',' ', sentence)
    sent_segment = ViTokenizer.tokenize(sentence)
    words = sent_segment.split()
    
    for word in words:
      if "_" in word:
        word_checking = " ".join(word.split("_"))
      else:
        word_checking = word

      if word_checking.lower() not in list_stopwords:
        parts.append(word)
    
    return " ".join(parts)



In [ ]:
dataset_test = pd.read_csv('/content/drive/MyDrive/Data/sentiment_analyst_data/DongDu_VN/test.csv')
dataset_train = pd.read_csv('/content/drive/MyDrive/Data/sentiment_analyst_data/DongDu_VN/train.csv')

In [ ]:
dataset_train

,Unnamed: 0,sentence,label
0,0,"Hôm_nay đi ăn tại quán , món ăn ngon vừa_miệng...",neg
1,1,Qua ́ n cư ̣ c ngon . Tuâ ̀ n na ̀ o mi ̀ nh c...,neg
2,2,"Chán , đồ uống khá nhạt . Với không_gian và ch...",neg
3,3,"Mì ăn ok 👍 🏻 👍 🏻 👍 🏻 nhưng nguội , view rộng ,...",neg
4,4,"Bữa có ghé đây ăn , súp chỉ bán tô , 1 tô 15k ...",neg
...,...,...,...
57821,57821,Chô ̃ gư ̉ i xe_hơi bâ ́ t tiê ̣ n đi khoa ̉ n...,pos
57822,57822,Quán nằm trong đường hơi khó tìm nhưng bù lại ...,pos
57823,57823,Mình dùng deal khi tới đây nhưng nhân_viên vẫn...,pos
57824,57824,Phần ăn của mình 2 người có_thể ăn no . Nó gồm...,pos


In [ ]:
path = '/content/drive/MyDrive/NLP/Clean Data/vietnamese.txt'
f = open(path, "r")
list_stopwords  = [word[0:-1] for word in f.readlines()]

In [ ]:
# dataset_train['tokens'] = dataset_train['tokens'].apply(lambda x: x[2:-2].split("', '"))
# dataset_train['tweet_cleaned'] = dataset_train['tokens'].apply(lambda x: " ".join(x))

In [ ]:
dataset_train['process_sent'] = dataset_train['sentence'].apply(lambda x: preprocess_sen(x))
dataset_test['process_sent'] = dataset_test['sentence'].apply(lambda x: preprocess_sen(x))

In [ ]:
dataset_train['label'] = dataset_train['label'].apply(lambda x: 1 if x == 'pos' else 0)
dataset_test['label'] = dataset_test['label'].apply(lambda x: 1 if x == 'pos' else 0)

In [ ]:
dataset_train

,Unnamed: 0,sentence,label,process_sent
0,0,"Hôm_nay đi ăn tại quán , món ăn ngon vừa_miệng...",0,"Hôm_nay đi ăn tại quán , món ăn ngon vừa_miệng..."
1,1,Qua ́ n cư ̣ c ngon . Tuâ ̀ n na ̀ o mi ̀ nh c...,0,Qua x n cư x c ngon . Tuâ x n na x o mi x nh c...
2,2,"Chán , đồ uống khá nhạt . Với không_gian và ch...",0,"Chán , đồ uống khá nhạt . Với không_gian và ch..."
3,3,"Mì ăn ok 👍 🏻 👍 🏻 👍 🏻 nhưng nguội , view rộng ,...",0,"Mì ăn ok 👍 🏻 👍 🏻 👍 🏻 nhưng nguội , view rộng ,..."
4,4,"Bữa có ghé đây ăn , súp chỉ bán tô , 1 tô 15k ...",0,"Bữa có ghé đây ăn , súp chỉ bán tô , 1 tô 15k ..."
...,...,...,...,...
57821,57821,Chô ̃ gư ̉ i xe_hơi bâ ́ t tiê ̣ n đi khoa ̉ n...,1,Chô x gư x i xe_hơi bâ x t tiê x n đi khoa x n...
57822,57822,Quán nằm trong đường hơi khó tìm nhưng bù lại ...,1,Quán nằm trong đường hơi khó tìm nhưng bù lại ...
57823,57823,Mình dùng deal khi tới đây nhưng nhân_viên vẫn...,1,Mình dùng deal khi tới đây nhưng nhân_viên vẫn...
57824,57824,Phần ăn của mình 2 người có_thể ăn no . Nó gồm...,1,Phần ăn của mình 2 người có_thể ăn no . Nó gồm...


In [ ]:
dataset_test

,Unnamed: 0,sentence,label,process_sent
0,0,"Cũng chả có gì đặc_biệt , chỉ đượ cái rẻ thôi ...",0,"Cũng chả có gì đặc_biệt , chỉ đượ cái rẻ thôi ..."
1,1,"Với_lại chỗ này không_gian quá hẹp , mà thiệt ...",0,"Với_lại chỗ này không_gian quá hẹp , mà thiệt ..."
2,2,Bánh flan truyền_thống ăn được nhất . Socola v...,0,Bánh flan truyền_thống ăn được nhất . Socola v...
3,3,"Về há cảo và xíu_mại thì cũng bình_thường , kh...",0,"Về há cảo và xíu_mại thì cũng bình_thường , kh..."
4,4,Chắc là giá bình_dân nên cũng không đòi_hỏi đư...,0,Chắc là giá bình_dân nên cũng không đòi_hỏi đư...
...,...,...,...,...
23163,23163,Mỳ và cơm ở đây rất ngon đặc_biệt là mấy loại ...,1,Mỳ và cơm ở đây rất ngon đặc_biệt là mấy loại ...
23164,23164,Cơm cực_kì ngon nhé thịt rất nhìu trộn đều và ...,1,Cơm cực_kì ngon nhé thịt rất nhìu trộn đều và ...
23165,23165,Bánh trông đẹp nhg ăn hk ngon lắm và giá khá m...,1,Bánh trông đẹp nhg ăn hk ngon lắm và giá khá m...
23166,23166,Nc uống ở đây nếu bạn hk gọi sẽ có trà đá free...,1,Nc uống ở đây nếu bạn hk gọi sẽ có trà đá free...


In [ ]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
dataset_train['token'] = dataset_train['process_sent'].apply(lambda x: word_tokenize(x))
dataset_test['token'] = dataset_test['process_sent'].apply(lambda x: word_tokenize(x))

In [ ]:
from collections import Counter
from tqdm import tqdm
from gensim.models import Word2Vec
import tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
datasets = dataset_train.append(dataset_test)
sequences_token = datasets['token'].values.tolist()
vocab = Counter()
for tokens in tqdm(sequences_token):
    vocab.update(tokens)

100%|██████████| 80994/80994 [00:01<00:00, 52725.05it/s]


In [ ]:
len(vocab)

50110

In [ ]:
def get_datasets(df, max_lenght, vocabsize = 30000):
  

  sequences_text = df['process_sent'].values.tolist()
  sequences_token = df['token'].values.tolist()
  y_target = np.array(df['label'].values.tolist())

 
  onehot_repr=[one_hot(words,vocabsize)for words in sequences_text]
  embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=max_lenght)

  X =np.array(embedded_docs)
  return X, y_target

In [ ]:
x_train, y_train = get_datasets(dataset_train, max_lenght = 70)
x_test, y_test = get_datasets(dataset_test, max_lenght = 70)
# y_train = to_categorical(targets, num_classes = 2)

## Embedding

### Word2Vec: Gensim

In [ ]:
model = Word2Vec(sequences_token, window=5, min_count=5, workers=16, sg=0, negative=5)

In [ ]:
word_vectors = model.wv
print("Number of word vectors: {}".format(len(word_vectors.vocab)))

Number of word vectors: 5688


In [ ]:
word_index = {t[0]: i+1 for i,t in enumerate(vocab.most_common(NUM_VOCAB))}

In [ ]:
# add 'UNK' -> unknow, 'PAD -> Padding in word indexx
word_index['UNK'] = 0
word_index['PAD'] = max(word_index.values()) + 1

In [ ]:
sequences = [[word_index.get(t, word_index['UNK']) for t in token]
             for token in sequences_token]
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post", value= word_index['PAD'])

In [ ]:
data[2]

In [ ]:
sequences_token[2]

['bihday', 'majesty']

In [ ]:
# bulding word matrix: dimension of matrix = number most fequency words's vocab -> word not in most word's vocab will random value

# we initialize the matrix with random numbers
wv_matrix = (np.random.rand(len(word_index), WV_DIM) - 0.5) / 5.0
for word, i in word_index.items():

    try:
        embedding_vector = word_vectors[word]
        # words not found in embedding index will be all-zeros.
        wv_matrix[i] = embedding_vector
    except:
        pass  

## Embedding: Onehot

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot
from sklearn.model_selection import train_test_split

In [ ]:

# Y_lstm = to_categorical(targets, num_classes = 2)
x_train, x_test, y_train, y_test = train_test_split(X_lstm, np.array(targets), test_size=0.33, random_state=42)

### Model Neural netword

In [ ]:
from tensorflow.keras import Sequential

from tensorflow.keras.layers import Embedding, LSTM,Bidirectional, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow


In [ ]:
vocabsize = 30000

In [ ]:
model = Sequential()
# model.add(Embedding(input_dim = len(word_index), output_dim = 100, embeddings_initializer = tensorflow.keras.initializers.Constant(wv_matrix), input_length= MAX_SEQUENCE_LENGTH))
model.add(Embedding(input_dim = vocabsize,output_dim = 50, input_length= MAX_SEQUENCE_LENGTH))

model.add(Bidirectional(LSTM(100,return_sequences=False)))
model.add(Dropout(0.3))

# model.add(LSTM(20,return_sequences=False))
# model.add(Dropout(0.3)) 

model.add(Dense(1, activation ='sigmoid'))
model.compile(loss="binary_crossentropy", optimizer= Adam(learning_rate=1e-2), metrics=['acc'])

In [ ]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=1000)

Epoch 1/10
58/58 [==============================] - 141s 2s/step - loss: 0.5713 - acc: 0.7221 - val_loss: 0.5288 - val_acc: 0.7564
Epoch 2/10
58/58 [==============================] - 114s 2s/step - loss: 0.3999 - acc: 0.8227 - val_loss: 0.4423 - val_acc: 0.7926
Epoch 3/10
58/58 [==============================] - 101s 2s/step - loss: 0.3531 - acc: 0.8480 - val_loss: 0.4447 - val_acc: 0.7930
Epoch 4/10
58/58 [==============================] - 98s 2s/step - loss: 0.3201 - acc: 0.8627 - val_loss: 0.4606 - val_acc: 0.7958
Epoch 5/10
58/58 [==============================] - 96s 2s/step - loss: 0.2814 - acc: 0.8822 - val_loss: 0.4739 - val_acc: 0.7954
Epoch 6/10
58/58 [==============================] - 96s 2s/step - loss: 0.2550 - acc: 0.8925 - val_loss: 0.5296 - val_acc: 0.7905
Epoch 7/10
58/58 [==============================] - 96s 2s/step - loss: 0.2307 - acc: 0.9034 - val_loss: 0.5369 - val_acc: 0.7871
Epoch 8/10
58/58 [==============================] - 97s 2s/step - loss: 0.2043 - acc: 0

In [ ]:
sequences_text = ['Anh sớm về với em :(((((']
onehot_repr=[one_hot(words,vocabsize)for words in sequences_text]
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=MAX_SEQUENCE_LENGTH)

X =np.array(embedded_docs)

In [ ]:
value = model.predict(X)[0]
if value > 0.5:
  print('day la mot cau tích cực')
else:
  print('câu tiêu cực')

day la mot cau tích cực


In [ ]:
model.save('/content/drive/MyDrive/NLP/model/my_model')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/NLP/model/my_model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/NLP/model/my_model/assets


In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report

In [ ]:
y_pred_lstm=model.predict(x_test)
y_pred_lstm = np.round(abs(y_pred_lstm))
print("Accuracy Score: ", accuracy_score(y_test, y_pred_lstm))
print("Confusion Matrix: \n", confusion_matrix(y_test,y_pred_lstm))
print(classification_report(y_test,y_pred_lstm))

Accuracy Score:  0.7837966160220995
Confusion Matrix: 
 [[8219 3253]
 [1756 9940]]
              precision    recall  f1-score   support

           0       0.82      0.72      0.77     11472
           1       0.75      0.85      0.80     11696

    accuracy                           0.78     23168
   macro avg       0.79      0.78      0.78     23168
weighted avg       0.79      0.78      0.78     23168



In [ ]:
! pip install gensim

### Embedding

In [ ]:
import numpy as np
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.test.utils import common_texts

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

from sklearn.linear_model import LogisticRegression

In [ ]:
# train word2vec on the two sentences
model = KeyedVectors.load_word2vec_format('/Users/phamvanmanh/Desktop/learn_python/tokenizer/gensim/wiki-news-300d-1M.vec')

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(sentences_vectonizer,targets,test_size=0.2,random_state=123)

### Logistic model

In [ ]:
df_results = pd.DataFrame(data=np.zeros(shape=(1,3)), columns = ['classifier', 'train_score', 'test_score'] )
train_score = clf.score(x_train, y_train)
test_score = clf.score(x_test, y_test)

df_results.loc[0,'classifier'] = "MLP"
df_results.loc[0,'train_score'] = train_score
df_results.loc[0,'test_score'] = test_score
 
print(df_results)

In [ ]:
### Neuron Network
from sklearn.neural_network import MLPClassifier

In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(x_train, y_train)

In [ ]:
df_results = pd.DataFrame(data=np.zeros(shape=(1,3)), columns = ['classifier', 'train_score', 'test_score'] )
train_score = clf.score(x_train, y_train)
test_score = clf.score(x_test, y_test)
 
 
df_results.loc[1,'classifier'] = "MLP"
df_results.loc[1,'train_score'] = train_score
df_results.loc[1,'test_score'] = test_score
 
print(df_results)

In [ ]:
sum(clf.predict(x_test))

### Modeling

In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
X, y = load_iris(return_X_y=True)
# clf = LogisticRegression(random_state=0).fit(X, y)
# clf.predict(X[:2, :])
# array([0, 0])
# clf.predict_proba(X[:2, :])
# array([[9.8...e-01, 1.8...e-02, 1.4...e-08],
#        [9.7...e-01, 2.8...e-02, ...e-08]])
# clf.score(X, y)

In [ ]:
sentences = [['this', 'is', 'the', 'good', 'machine', 'learning', 'book'],
            ['this', 'is',  'another', 'machine', 'learning', 'book'],
            ['one', 'more', 'new', 'book'],
         
          ['this', 'is', 'about', 'machine', 'learning', 'post'],
          ['orange', 'juice', 'is', 'the', 'liquid', 'extract', 'of', 'fruit'],
          ['orange', 'juice', 'comes', 'in', 'several', 'different', 'varieties'],
          ['this', 'is', 'the', 'last', 'machine', 'learning', 'book'],
          ['orange', 'juice', 'comes', 'in', 'several', 'different', 'packages'],
          ['orange', 'juice', 'is', 'liquid', 'extract', 'from', 'fruit', 'on', 'orange', 'tree']]


In [ ]:
import numpy as np
 

 
 
V=[]
for sentence in sentences:
    V.append(sent_vectorizer(sentence, model)) 

NameError: ignored

In [ ]:
V[0]